# Homework and bakeoff: Few-shot OpenQA with DSPy

In [4]:
__author__ = "Christopher Potts and Omar Khattab"
__version__ = "CS224u, Stanford, Fall 2024"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cgpotts/cs224u/blob/master/hw_openqa.ipynb)

If Colab is opened with this badge, please **save a copy to drive** (from the File menu) before running the notebook.

## Overview

The goal of this homework is to explore retrieval-augmented in-context learning. This is an exciting area that brings together a number of recent task ideas and modeling innovations. We will use the [DSPy programming library](http://dspy.ai) to build systems in this new mode.

Our core task is __open-domain question answering (OpenQA)__. In this task, all that is given by the dataset is a question text, and the task is to answer that question. By contrast, in many modern QA tasks, the dataset provides a text and a gold passage, usually with a firm guarantee that the answer will be a substring of the passage.

OpenQA is substantially harder than standard QA. The usual strategy is to use a _retriever_ to find passages in a large collection of texts and train a _reader_ to find answers in those passages. This means we have no guarantee that the retrieved passage will contain the answer we need. If we don't retrieve a passage containing the answer, our reader has no hope of succeeding. Although this is challenging, it is much more realistic and widely applicable than standard QA. After all, with the right retriever, an OpenQA system could be deployed over the entire Web.

The task posed by this homework is harder even than OpenQA. We are calling this task __few-shot OpenQA__. The defining feature of this task is that the reader is simply a frozen, general purpose language model. It accepts string inputs (prompts) and produces text in response. It is not trained to answer questions per se, and nothing about its structure ensures that it will respond with a substring of the prompt corresponding to anything like an answer.

__Few-shot QA__ (but not OpenQA!) is explored in the famous GPT-3 paper ([Brown et al. 2020](https://arxiv.org/abs/2005.14165)). The authors are able to get traction on the problem using GPT-3, an incredible finding. Our task here – __few-shot OpenQA__ – pushes this even further by retrieving passages to use in the prompt rather than assuming that the gold passage can be used in the prompt. If we can make this work, then it should be a major step towards flexibly and easily deploying QA technologies in new domains.

In summary:

| Task             | Passage given | Task-specific reader training |Task-specific retriever training  | 
|-----------------:|:-------------:|:-----------------------------:|:--------------------------------:|
| QA               | yes           | yes                           | n/a                              |
| OpenQA           | no            | yes                           | maybe                            |
| Few-shot QA      | yes           | no                            | n/a                              |
| Few-shot OpenQA  | no            | no                            | maybe                            | 

Just to repeat: your mission is to explore the final line in this table. The core notebook and assignment don't address the issue of training the retriever in a task-specific way, but this is something you could pursue for a final project; [the ColBERT codebase](https://github.com/stanford-futuredata/ColBERT) makes easy.

It is a requirement of the bake-off that a general-purpose language model be used. In particular, trained QA systems cannot be used at all, and no fine-tuning is allowed either. See the original system question at the bottom of this message for guidance on which models are allowed.

Note: the models we are working with here are _big_. This poses a challenge that is increasingly common in NLP: you have to pay one way or another. You can pay to use the GPT-3 API, or you can pay to use a local model on a heavy-duty cluster computer, or you can pay with time by using a local model on a more modest computer.

## Set-up

We have sought to make this notebook self-contained and easy to use on a personal computer, on Google Colab, and in Sagemaker Studio. For personal computer use, we assume you have already done everything in [setup.ipynb](setup.ipynb]). For cloud usage, the next few code blocks should handle all set-up steps.

In [ ]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import datasets
    root_path = '.'
except ModuleNotFoundError:
    !git clone https://github.com/cgpotts/cs224u/
    !pip install -r cs224u/requirements.txt
    root_path = 'dspy'

In [5]:
from datasets import load_dataset
import os
import dspy
import warnings

from dotenv import load_dotenv

### OpenAI set-up

In [6]:
from openai import OpenAI

In [197]:
root_path = '/Users/mingbosha/Documents'
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')


# keep the API keys in a `.env` file in the local root directory
load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')  # use the .env file as it is a good practice to keep keys outside of one's code

### ColBERT set-up

#### Pretrained ColBERT parameters

For this set-up phase, we first download pretrained ColBERT parameters:

In [18]:
if not os.path.exists(os.path.join("data", "openqa", "colbertv2.0.tar.gz")):
    !mkdir -p data/openqa
    # ColBERTv2 checkpoint trained on MS MARCO Passage Ranking (388MB compressed)
    !wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz -P data/openqa/
    !tar -xvzf data/openqa/colbertv2.0.tar.gz -C data/openqa/

--2025-03-17 17:08:59--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
connected. to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 405924985 (387M) [application/octet-stream]
Saving to: ‘data/openqa/colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  2.07MB/s    in 80s     

2025-03-17 17:10:19 (4.82 MB/s) - ‘data/openqa/colbertv2.0.tar.gz’ saved [405924985/405924985]

x colbertv2.0/
x colbertv2.0/artifact.metadata
x colbertv2.0/vocab.txt
x colbertv2.0/tokenizer.json
x colbertv2.0/special_tokens_map.json
x colbertv2.0/tokenizer_config.json
x colbertv2.0/config.json
x colbertv2.0/pytorch_model.bin


If something went wrong with the above, you can just download the file https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz, unarchive it, and move the resulting `colbertv2.0` directory into the `data/openqa` directory.

#### Prebuilt ColBERT index

Then we download a prebuilt index that covers the domain of our task:

In [19]:
index_home = os.path.join("experiments", "notebook", "indexes", "cs224u.collection.2bits")

In [20]:
if not os.path.exists(index_home):
    !wget https://web.stanford.edu/class/cs224u/data/cs224u.collection.2bits.tgz -P experiments/notebook/indexes
    !tar -xvzf experiments/notebook/indexes/cs224u.collection.2bits.tgz -C experiments/notebook/indexes

--2025-03-17 17:12:12--  https://web.stanford.edu/class/cs224u/data/cs224u.collection.2bits.tgz
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200
connected. to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... 
200 OKequest sent, awaiting response... 
Length: 600150346 (572M) [application/x-gzip]
Saving to: ‘experiments/notebook/indexes/cs224u.collection.2bits.tgz’

cs224u.collection.2 100%[===================>] 572.35M   599KB/s    in 19m 58s 

2025-03-17 17:32:10 (489 KB/s) - ‘experiments/notebook/indexes/cs224u.collection.2bits.tgz’ saved [600150346/600150346]

x cs224u.collection.2bits/
x cs224u.collection.2bits/buckets.pt
x cs224u.collection.2bits/avg_residual.pt
x cs224u.collection.2bits/3.codes.pt
x cs224u.collection.2bits/4.codes.pt
x cs224u.collection.2bits/4.residuals.pt
x cs224u.collection.2bits/metadata.json
x cs224u.collection.2bits/doclens.4.json
x cs224u.collection.2bits/ivf.pt
x cs224u.collection.2bits/3.metadata.json
x cs224u.collection.2bits/1.c

If something went wrong with the above, download the file https://web.stanford.edu/class/cs224u/data/cs224u.collection.2bits.tgz, unarchive it, and move the resulting `cs224u.collection.2bits` directory into the `experiments/notebook/indexes` directory (which you will probably need to create).

#### ColBERT server

The final step for ColBERT is to create a local retriever for use with DSPy. The preferred method for doing this is with a seperate server. 

__Local usage__: If you are working with this notebook locally and a having a CUDA based device, make sure you have the [CUDA Toolkit](https://developer.nvidia.com/cuda-downloads) installed to be able to serve it on GPU. Otherwise you would need to run it on CPU. The next steps are for GPU usage. 
Open a new terminal window, navigate to the same directory as this notebook, and enter the following code:

```
conda activate nlu
git clone https://github.com/stanford-futuredata/ColBERT/ ;
export INDEX_ROOT=experiments/notebook/indexes/cs224u.collection.2bits/ ;
export INDEX_HOME=cs224u.collection.2bits ;
export PORT=8888 ;
python ColBERT/server.py 
```

This will start the server and you should be all set.

__Colab usage__: If you are working in a Colab _and you have a Pro account_, you can open a terminal from the lower left corner of the interface and then paste in the above code. If you don't have a Pro account, then it should work to uncomment the code in the following cell and run it:

In [ ]:
#os.environ["INDEX_ROOT"] = "experiments/notebook/indexes/cs224u.collection.2bits/"
#os.environ["INDEX_HOME"] = "cs224u.collection.2bits"
#os.environ["PORT"] = "8888"

#!git clone https://github.com/stanford-futuredata/ColBERT/

#!nohup python ColBERT/server.py &

This will take a minute to get started, and you won't get feedback on its progress, unfortunately. You can check that a server is running:

In [ ]:
!ps aux | grep server.py

You should see a mention of `ColBERT/server.py` in the output. 

Assuming the server is now running, we can connect to it:

*Note*: if the ColBERT server is running on a separate server, you can update `127.0.0.1` with the server's IP address.

In [60]:
rm = dspy.ColBERTv2(url="http://10.0.0.38:8889//api/search")

In [208]:
from dspy.retrieve.you_rm import YouRM
import os

you_rm = YouRM(endpoint="search", safesearch="strict")

results = you_rm("how many colors are there for iPhone 16 Pro Max?", k=5)

for result in results:
    print(result)


{'long_text': 'This is basically just a shade of white, which itself is a very popular, common, and (whisper it) ordinary smartphone color. So this is likely to sell well and unlikely to turn heads, but fitting in is sometimes desirable. ... Finally, there’s a Black Titanium shade, which has a hint of gray to it but is by far the darkest shade that you can get the iPhone 16 Pro and iPhone 16 Pro Max in.'}
{'long_text': "Natural Titanium is pleasant but a but basic, and arguably lacks the punch Apple's classic Space Gray color offers. White Titanium really caught our eye, which when combined with the black of the rear cameras, give the iPhone 16 Pro and 16 Pro Max a Star Wars Stormtrooper look, which we rather like and therefore recommend the most."}
{'long_text': 'The iPhone 16 Pro and iPhone 16 Pro Max excel in numerous ways, from powerful chipsets to capable cameras and beyond, but do their colors impress? Typically, Apple just offers a small number of color choices for its Pro phone

### DSPy set-up

Here we establish the Language Model `lm` and Retriever Model `rm` that we will be using. The defaults for `lm` are just for development. You may want to develop using an inexpensive model and then do your final evalautions wih an expensive one. DSPy has support for a wide range of model APIs and local models.

In [61]:
lm = dspy.OpenAI(model='gpt-3.5-turbo', api_key=openai_key)

dspy.settings.configure(lm=lm, rm=rm)

Here's a command you can run to see which OpenAI models are available; OpenAI has entered into an increasingly closed mode where many older models are not available, so there are likely to be some surprises lurking here:

In [26]:
# client = OpenAI(api_key = openai_key)
# models = client.models.list()
# print(models)

## SQuAD

Our core development dataset is [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/). We chose this dataset because it is well-known and widely used, and it is large enough to support lots of meaningful development work, without, though, being so large as to require lots of compute power.

In [27]:
squad = load_dataset("squad", trust_remote_code=True)

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

The following utility just reads a SQuAD split in as a list of `dspy.Example` instances:

In [28]:
def get_squad_split(squad, split="validation"):
    """
    Use `split='train'` for the train split.

    Returns
    -------
    list of dspy.Example with attributes question, answer

    """
    data = zip(*[squad[split][field] for field in squad[split].features])
    exs = [dspy.Example(question=q, answer=a['text'][0]).with_inputs("question")
           for eid, title, context, q, a in data]
    return exs

### SQuAD train

To build few-shot prompts, we will often sample SQuAD train examples, so we load that split here:

In [29]:
squad_train = get_squad_split(squad, split="train")

### SQuAD dev

In [30]:
squad_dev = get_squad_split(squad)

### SQuAD dev sample

Evaluations are expensive in this new era! Here's a small sample to use for dev assessments:

In [31]:
import random

random.seed(1)

dev_exs = random.sample(squad_dev, k=200)

## DSPy basics

### LM usage

Here's the most basic way to use the LM:

In [33]:
lm("What is the birthplace of the first author to win a Hugo Award for a translation?")

['The birthplace of the first author to win a Hugo Award for a translation is Japan. The author is Yoshio Kobayashi, who won the Hugo Award for Best Novel in 1980 for his translation of "The Left Hand of Darkness" by Ursula K. Le Guin.']

Keyword arguments to the underlying LM are passed through:

In [34]:
lm("Which U.S. states border no U.S. states?", temperature=0.9, n=4)

['There are two states that do not border any other U.S. state: Alaska and Hawaii.',
 'There are five U.S. states that do not border any other U.S. states. They are:\n\n1. Alaska\n2. Hawaii\n3. Washington D.C.\n4. Delaware\n5. Rhode Island',
 'Maine and Hawaii are the only two U.S. states that do not share a border with any other U.S. state.',
 'There are three U.S. states that do not share a border with any other U.S. states: Alaska, Hawaii, and Rhode Island.']

With `lm.inspect_history`, we can see the most recent language model calls:

In [35]:
_ = lm.inspect_history(n=1)




Which U.S. states border no U.S. states? There are two states that do not border any other U.S. state: Alaska and Hawaii. 	 (and 3 other completions)





### Signature-based prediction

In DSPy, __signatures__ are declarative statements about what we want the model to do. In the following `"question -> answer"` is the signature (the most basic QA signature one could write), and `dspy.Predict` is used to turn this into a complete QA system: 

In [36]:
basic_predictor = dspy.Predict("question -> answer")

Here we use `basic_predictor`:

In [37]:
basic_predictor(question="What is the birthplace of the first author to win a Hugo Award for a translation?")

Prediction(
    answer='Paris, France'
)

And here is the prompt that was given to the model:

In [38]:
_ = lm.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: What is the birthplace of the first author to win a Hugo Award for a translation?
Answer: Paris, France





In many cases, we will want more control over the prompt. Writing a small custom `dspy.Signature` class is the easiest way to accomplish this. In the following, we just just tweak the initial instruction and provide some formatting guidance for the answer:

In [39]:
class BasicQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [40]:
sig_predictor = dspy.Predict(BasicQASignature)

In [41]:
sig_predictor(question="Which U.S. states border no U.S. states?")

Prediction(
    answer='Maine, Hawaii'
)

In [42]:
_ = lm.inspect_history(n=1)




Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: Which U.S. states border no U.S. states?
Answer: Maine, Hawaii





### Modules

One of the hallmarks of DSPy is that it adopts design patterns from PyTorch. The main example of this is DSPy's use of the `Module` as the basic unit for writing simple and complex programs. Here is a very basic module for QA that makes use of `BasicQASignature` as we defined it just above.

In [43]:
class BasicQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict(BasicQASignature)

    def forward(self, question):
        return self.generate_answer(question=question)

As with PyTorch, the `forward` method is called when we want to make predictions:

In [44]:
basic_qa_model = BasicQA()

In [45]:
basic_qa_model(question="What is the birthplace of the first author to win a Hugo Award for a translation?")

Prediction(
    answer='Poland'
)

The modular design of DSPy starts to become apparent now. If you want to change the above to use chain of thought instead of regular predictions, you need only change `dspy.Predict` to `dspy.ChainOfThought`, and similarly for `dspy.ReAct`, `dspy.ProgramOfThought`, or a module you wrote yourself.

### Optimizing

The QA system we've defined so far is a zero-shot system. To change it into a few-shot system, we will rely on a DSPy optimizer (__teleprompter__). This will allow us to flexibly move between the zero-shot and few-shot formulations. The following code achieves this.

In [48]:
from dspy.teleprompt import LabeledFewShot

/Users/mingbosha/Documents/cache/compiler


Here we instantiate a `LabeledFewShot` teleprompter that will add three demonstrations. These will be sampled randomly from the set of train examples we provide:

In [49]:
fewshot_teleprompter = LabeledFewShot(k=3)

And then we call `compile` on `basic_qa_model` as we defined it above. This returns a new module that we use like any other in DSPy:

In [50]:
basic_fewshot_qa_model = fewshot_teleprompter.compile(basic_qa_model, trainset=squad_train)

In [51]:
basic_fewshot_qa_model(question="What is the birthplace of the first author to win a Hugo Award for a translation?")

Prediction(
    answer='China'
)

With `inspect_history`, we can see that prompts now contain demonstrations:

In [119]:
_ = lm.inspect_history(n=3)




Answer questions with short factoid answers.

---

Follow the following format.

Context: may contain relevant facts
Question: ${question}
Answer: often between 1 and 5 words

---

Question: What are new responsibilities pharmacy technicians now deal with?
Answer: patients' prescriptions and patient safety issues

Question: What do capitalist firms substitute equipment for in a Marxian analysis?
Answer: labor inputs

Question: Which building was the NFL Experience held at for Super Bowl 50?
Answer: Moscone Center

Question: In South Africa, along with privately governed schools, what schools are classified as independent?
Answer: traditional private

Question: In 1517 who was Luther's bishop?
Answer: Albert of Mainz

Question: What would a teacher do for someone who is cocky?
Answer: deflate

Question: Who continued to hold the citadel of Bukhara after the Mongols took the rest of the city?
Answer: a unit of Turkish defenders

Question: In what year did the film also mention the num

### Evaluation

Our evaluation metric is a standard one for SQuAD and related tasks: exact match of the answer (EM).

In [53]:
from dspy.evaluate import answer_exact_match
from dspy.evaluate.evaluate import Evaluate

In [54]:
answer_exact_match(dspy.Example(answer="STAGE 2!"), dspy.Prediction(answer="stage 2"))

True

In DSPy, `Evaluate` objects provide a uniform interface for running evaluations. Here are two for us to use in development. The first will evaluate on all of `dev_exs` and should provide a meaningful picture of how a system is doing. It could be expensive to use it a lot, though. The second is for debugging and is probably too small to give a reliable estimate.

In [ ]:
dev_evaluater = Evaluate(
    devset=dev_exs, # 200 examples
    num_threads=1,
    display_progress=True,
    display_table=5)

In [55]:
tiny_evaluater = Evaluate(
    devset=dev_exs[: 15],
    num_threads=1,
    display_progress=True,
    display_table=5)

Here is a tiny (debugging-oriented) evaluation of our few-shot QA sytem:

In [56]:
tiny_evaluater(basic_fewshot_qa_model, metric=answer_exact_match)

Average Metric: 2 / 15  (13.3): 100%|███████████████████████████████| 15/15 [00:05<00:00,  2.63it/s]


,question,example_answer,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,Albert of Mainz,✔️ [True]
1,When was the construction that changed the Rhine's Delta?,20th Century,13th century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","over 100,000",False
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",interpret EU law,False
4,What would a teacher do for someone who is cocky?,deflate,ignore them,False


13.33

### Retrieval

The final major component of our systems is retrieval. When we defined `rm`, we connected to a remote ColBERT index and retriever system that we can now use for search.

The basic `dspy.retrieve` method returns only passages:

In [210]:
retriever = dspy.Retrieve(k=3)

In [63]:
passages = retriever("What is the birthplace of the first author to win a Hugo Award for a translation?")

In [70]:
passages = retriever("How much are the iPhone X")

In [71]:
passages.passages[0]

'Macintosh | US$29.99, it allowed adventurous Mac users to sample Apple\'s new operating system and provide feedback for the actual release. The initial version of Mac OS X, 10.0 "Cheetah", was released on March 24, 2001. Older Mac OS applications could still run under early Mac OS X versions, using an environment called "Classic". Subsequent releases of Mac OS X included 10.1 "Puma" (2001), 10.2 "Jaguar" (2002), 10.3 "Panther" (2003) and 10.4 "Tiger" (2005). Apple discontinued the use of PowerPC microprocessors in 2006. At WWDC 2005, Steve Jobs announced this transition, revealing that Mac OS X was always developed to run on'

In [194]:
rm("How much are the iPhone 8?", k=3)

[{'pid': 48656,
  'prob': 0.9589276606710758,
  'rank': 1,
  'score': 22.34375,
  'text': 'IPhone | revenue (until the iPhone 3G), in exchange for four years of exclusive US sales, until 2011. Jobs unveiled the iPhone to the public on January 9, 2007, at the Macworld 2007 convention at the Moscone Center in San Francisco. The two initial models, a 4\xa0GB model priced at US$499 and an 8\xa0GB model at US$599 (both requiring a 2-year contract), went on sale in the United States on June 29, 2007, at 6:00\xa0pm local time, while hundreds of customers lined up outside the stores nationwide. The passionate reaction to the launch of the iPhone resulted in sections of the media',
  'long_text': 'IPhone | revenue (until the iPhone 3G), in exchange for four years of exclusive US sales, until 2011. Jobs unveiled the iPhone to the public on January 9, 2007, at the Macworld 2007 convention at the Moscone Center in San Francisco. The two initial models, a 4\xa0GB model priced at US$499 and an 8\xa0

If we need passages with scores and other metadata, we can call `rm` directly:

In [65]:
rm("What is the birthplace of the first author to win a Hugo Award for a translation?", k=1)

[{'pid': 30459,
  'prob': 1.0,
  'rank': 1,
  'score': 22.015625,
  'text': 'Ken Liu | Problem" won the 2015 Hugo Award, the first translated novel in the award\'s history to have won that honor. Ken Liu is currently writing his series "The Dandelion Dynasty" (edited by Joe Monti) for Saga Press. The first novel in the series, "The Grace of Kings", was a 2016 Nebula Award finalist. His official Star Wars novel called "The Legends of Luke Skywalker" was published October 31, 2017. Liu was born in 1976 in Lanzhou, China, and emigrated to the United States when he was 11 years old, initially living in Palo Alto, California, and later moving to Waterford, Connecticut.',
  'long_text': 'Ken Liu | Problem" won the 2015 Hugo Award, the first translated novel in the award\'s history to have won that honor. Ken Liu is currently writing his series "The Dandelion Dynasty" (edited by Joe Monti) for Saga Press. The first novel in the series, "The Grace of Kings", was a 2016 Nebula Award finalist. H

## Few-shot OpenQA with context

Let's build on the above core concepts to define a basic retrieval-augmented generation (RAG) program. This program solves the core task of few-shot OpenQA task and will serve as the basis for the homework questions:

We begin with a signature that takes context into account but is otherwise just like `BasicQASignature` above:

In [75]:
class ContextQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

And here is a complete program/system for the task:

In [76]:
class RAG(dspy.Module):
    def __init__(self, num_passages=1):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.Predict(ContextQASignature)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [77]:
rag_model = RAG(num_passages=3)

In [78]:
rag_model(question="What is the birthplace of the first author to win a Hugo Award for a translation?")

Prediction(
    context=['Ken Liu | Problem" won the 2015 Hugo Award, the first translated novel in the award\'s history to have won that honor. Ken Liu is currently writing his series "The Dandelion Dynasty" (edited by Joe Monti) for Saga Press. The first novel in the series, "The Grace of Kings", was a 2016 Nebula Award finalist. His official Star Wars novel called "The Legends of Luke Skywalker" was published October 31, 2017. Liu was born in 1976 in Lanzhou, China, and emigrated to the United States when he was 11 years old, initially living in Palo Alto, California, and later moving to Waterford, Connecticut.', 'Vernor Vinge | Vernor Vinge Vernor Steffen Vinge (; born October 2, 1944) is an American science fiction author and retired professor. He taught mathematics and computer science at San Diego State University. He is the originator of the technological singularity concept and perhaps the first to present a fictional "cyberspace". He has won the Hugo Award for his novels and 

An optional tiny evaluation:

In [79]:
tiny_evaluater(rag_model, metric=answer_exact_match)

Average Metric: 8 / 15  (53.3): 100%|███████████████████████████████| 15/15 [00:07<00:00,  1.93it/s]


,question,example_answer,context,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"['Martin Luther | St. Peter\'s Basilica in Rome. Roman Catholic theology stated that faith alone, whether fiduciary or dogmatic, cannot justify man; justification rather depends...",Albert of Mainz,✔️ [True]
1,When was the construction that changed the Rhine's Delta?,20th Century,"['Rhine | characterized by the delta\'s main arms, disconnected arms (Hollandse IJssel, Linge, Vecht, etc.) and smaller rivers and streams. Many rivers have been closed...",second half of the 20th Century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","['Warsaw | a ""major world city"") by the Globalization and World Cities (GaWC) Study Group and Network from Loughborough University, placing it on a par...","304,016",✔️ [True]
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['European Union law | judges for three years. While TEU article 19(3) says the Court of Justice is the ultimate court to interpret questions of...,"""ensure that in the interpretation and application of the Treaties the law is observed""",False
4,What would a teacher do for someone who is cocky?,deflate,"['Teacher | described the place of a teacher in learning as follows: ""The real bulk of learning takes place in self-study and problem solving with...",deflate,✔️ [True]


53.33

## Question 1: Optimizing RAG [2 points]

We used `RAG` above as a zero-shot system. We could turn it into a few-shot system by using `LabeledFewShot` as we did in [the teleprompting section](#Teleprompting) above, but this may actually be problematic: if we randomly sample demonstrations with retrieved passages, we might be instructing the model with a lot of cases where the context passage isn't helping (and may actually be actively misleading the model). 

What we'd like to do is select demonstrations where the model gets the answer correct and the context passage does contain the answer. To do this, we will use the DSPy `BootstrapFewShot` optimizer. There are two steps for this: (1) defining a metric and (2) running the optimizer.

__Note__: The code for this question can be found in the DSPy tutorials, and you should feel free to make use of that code. The goal is to help you understand the design patterns and overall logic of optimizing DSPy programs.

__Task 1__: Complete `validate_context_and_answer` according to the specification in the docstring.

In [92]:
def validate_context_and_answer(example, pred, trace=None):
    """Return True if `example.answer` matches `pred.answer` according
    to `dspy.evaluate.answer_exact_match` and `pred.context` contains
    `example.answer` according to `dspy.evaluate.answer_passage_match`.

    Parameters
    ----------
    example: dspy.Example 
        with attributes `answer` and `context`
    pred: dspy.Example 
        with attributes `answer` and `context`
    trace : None (included for dspy internal compatibility)

    Returns
    -------
    bool

    """
    pass
    ##### YOUR CODE HERE
    from dspy.evaluate import answer_exact_match, answer_passage_match
    answer_match = answer_exact_match(example, pred)   
    context_match = answer_passage_match(example, pred)
    
    return answer_match and context_match




A test you can use to check your implementation:

In [93]:
def test_validate_context_and_answer(func):
    examples = [
        (
            dspy.Example(question="Q1", answer="B"),
            dspy.Prediction(question="Q1", context="A B C", answer="B"),
            True
        ),
        # Context doesn't contain answer, but predicted answer is correct.
        (
            dspy.Example(question="Q1", answer="D"),
            dspy.Prediction(question="Q1", context="A B C", answer="D"),
            False
        ),
        # Context contains answer, but predicted answer is not correct.
        (
            dspy.Example(question="Q1", answer="C"),
            dspy.Prediction(question="Q1", context="A B C", answer="D"),
            False
        )
    ]
    errcount = 0
    for ex, pred, result in examples:
        predicted = func(ex, pred, trace=None)
        if predicted != result:
            errcount += 1
            print(f"Error for `{func.__name__}`: "
                  f"Expected inputs\n\t{ex}\n\t{pred} to return {result}.")
    if errcount == 0:
        print(f"No errors detected for `{func.__name__}`")

In [94]:
test_validate_context_and_answer(validate_context_and_answer)

No errors detected for `validate_context_and_answer`


__Task 2__: Complete `bootstrap_optimize` according to the specification in the docstring.

In [126]:
from dspy.teleprompt import BootstrapFewShot

def bootstrap_optimize(model):
    """Use `BootstrapFewShot` to optimize `model`, with the metric set
    to `validate_context_and_answer` as defined above and default
    values for all other keyword arguments to `BootstrapFewShot`.

    Parameters
    ----------
    model: dspy.Module

    Returns
    -------
    dspy.Module, the optimized version of `model`

    """
    pass
    ##### YOUR CODE HERE
    optimizer = BootstrapFewShot(metric=validate_context_and_answer)
    optimized_model = optimizer.compile(model, trainset=dev_exs[: 15])
    
    return optimized_model



A test you can use to check your implementation:

In [127]:
def test_bootstrap_optimize(func):
    model = RAG()
    compiled = func(model)
    if not hasattr(compiled, "_compiled") or not compiled._compiled:
        print(f"Error for `{func.__name__}`: "
               "The return value is not a compiled program.")
        return None
    state = compiled.dump_state()
    if not state['generate_answer']['demos']:
        print(f"Error for `{func.__name__}`: "
               "The compiled program has no `demos`.")
        return None
    print(f"No errors detected for `{func.__name__}`")

In [128]:
test_bootstrap_optimize(bootstrap_optimize)

 40%|█████████████████████████▏                                     | 6/15 [00:00<00:00, 625.66it/s]

Bootstrapped 4 full traces after 7 examples in round 0.
[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x3306573d0>), ('generate_answer', Predict(ContextQASignature(context, question -> answer
    instructions='Answer questions with short factoid answers.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]
No errors detected for `bootstrap_optimize`


## Question 2: Multi-passage summarization [2 points]

The `dspy.Retrieve` layer in our `RAG` retrieves `k` passages, where `k` is under the control of the user. One hypothesis one might have is that it would be good to summarize these passages before using them as evidence. This seems especially likely to help in scenarios where the question can be answered only by synthesizing information across documents – it might be too much to ask the language model to do both synthesizing and answering in a single step.

The current question maps out a basic strategy for summarization. The heart of it is a new signature called `SummarizeSignature`. This can be used on its own with a simple `dspy.Predict` call, and we'll incorporate it into a RAG program in the next question.

For this question, though, your task is just to complete `SummarizeSignature`. The requirements are as follows:

1. A `__doc__` value that gives an instruction that seems to work well. You can decide what to say here.
2. A `dspy.InputField` named `context`. You can decide whether to use the `desc` parameter.
3. A `dspy.OutputField` named `summary`. You can decide whether to use the `desc` parameter.

In [142]:
class SummarizeSignature(dspy.Signature):
    pass
    ##### YOUR CODE HERE
    __doc__ = """Summarize the key information of the provided context into 4-5 sentences"""

    context = dspy.InputField()
    summary = dspy.OutputField()




Here's a simple test that just checks for the required pieces in a basic way:

In [144]:
def test_SummarizeSignature(sigclass):
    fields = sigclass.fields
    expected_fieldnames = ['context', 'summary']
    fieldnames = sorted(fields)
    errcount = 0
    if expected_fieldnames != fieldnames:
        errcount += 1
        print(f"Error for `{sigclass.__name__}`: "
              f"Expected fieldnames {expected_fieldnames}, got {fieldnames}.")
    if not sigclass.__doc__:
        errcount += 1
        print(f"Error for `{sigclass.__name__}`: No docstring specified.")
    if errcount == 0:
        print(f"No errors detected for `{sigclass.__name__}`")

In [145]:
test_SummarizeSignature(SummarizeSignature)

No errors detected for `SummarizeSignature`


Here is the simplest way to use `SummarizeSignature`:

In [146]:
summarizer = dspy.Predict(SummarizeSignature)

In [182]:
summarizer(context=retriever("Where is Guarani spoken?").passages)

'The context discusses the presence of indigenous languages in Latin America, with languages like Quichua, Aymara, Quechua, Guaraní, and Guarani being recognized and even holding official status in countries like Ecuador, Bolivia, Paraguay, and Argentina. Additionally, the text mentions the presence of other languages in Argentina, such as Standard German, Yiddish, Catalan, Mapudungun, and Chinese. It also highlights the diversity of indigenous languages spoken in the Americas, with over a thousand different languages being spoken, including Quechuan languages, Aymara, Guaraní, Mayan languages, and Nahuatl. Despite the influence of Western culture, many indigenous peoples still maintain traditional aspects of their culture while adapting'

In [149]:
_ = lm.inspect_history(n=1)




Summarize the key information of the provided context into 4-5 sentences

---

Follow the following format.

Context: ${context}
Summary: ${summary}

---

Context:
[1] «Latin Americans | alongside Spanish and any other indigenous language in the areas where they predominate. In Ecuador, while holding no official status, the closely related Quichua is a recognized language of the indigenous people under the country's constitution; however, it is only spoken by a few groups in the country's highlands. In Bolivia, Aymara, Quechua and Guaraní hold official status alongside Spanish. Guarani is, along with Spanish, an official language of Paraguay, and is spoken by a majority of the population (who are, for the most part, bilingual), and it is co-official with Spanish in the Argentine province of Corrientes. In Nicaragua,»
[2] «Languages of Argentina | is classified as a Quechua II language and is referred to as Quechua IIC by linguists. Standard German is spoken by between 400,000 and 50

## Question 3: Summarizing RAG [2 points]

Your task for this question is to modify `RAG` as defined above so that the retrieved passages are summarized before being passed to `generate_answer`. 

Here is the `RAG` system copied from above with the class name changed to the one we will use for this new system. Your task is to add the summarization step. This should be very straightforward given the modular design that DSPy supports and encourages!

In [186]:
class SummarizingRAG(dspy.Module):
    def __init__(self, num_passages=3):
        # Please name your summarization later `summarize` so that we
        # can check for its presence.
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        ##### YOUR CODE HERE
        self.summarize = dspy.Predict(SummarizeSignature)

        self.generate_answer = dspy.Predict(ContextQASignature)

    def forward(self, question):
        context = self.retrieve(question).passages
        ##### YOUR CODE HERE
        context = [self.summarize(context=c).summary for c in context]

        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

A simple test for this design spec:

In [187]:
def test_SummarizingRAG(classname):
    model = classname(num_passages=3)
    errcount = 0
    if not hasattr(model, "summarize"):
        errcount += 1
        print(f"Error for `{classname.__name__}`: "
              f"Expected a layer called 'summarize'")
    context = model.retrieve("What are some foods?").passages
    pred = model("What are some foods?")
    if context == pred.context:
        errcount += 1
        print(f"Error for `{classname.__name__}`: "
              "The model seems to be using raw retrieved contexts "
              "for predictions rather than summarizing them.")
    if errcount == 0:
        print(f"No errors detected for `{classname.__name__}`")

In [188]:
test_SummarizingRAG(SummarizingRAG)

No errors detected for `SummarizingRAG`


Model usage:

In [189]:
summarizing_rag_model = SummarizingRAG()

In [190]:
summarizing_rag_model(question="What is the birthplace of the first author to win a Hugo Award for a translation?")

Prediction(
    context=['Ken Liu, a Chinese-American author, made history by winning the 2015 Hugo Award for his novel "The Three-Body Problem," the first translated work to do so. He is known for his series "The Dandelion Dynasty," with the first book, "The Grace of Kings," being a Nebula Award finalist. Liu also wrote an official Star Wars novel titled "The Legends of Luke Skywalker." Born in 1976 in China, Liu immigrated to the United States at the age of 11, settling in California and later Connecticut.', 'Vernor Vinge is an American science fiction author and former professor known for his work on the technological singularity concept and cyberspace. He has won the Hugo Award for several of his novels and novellas, including "A Fire Upon the Deep" and "Rainbows End". Vinge began his writing career with the publication of his short story "Bookworm, Run!" in 1966. He taught mathematics and computer science at San Diego State University.', 'Context: Translation | Awards annually pre

In [226]:
_ = lm.inspect_history(n=1)




Answer questions with short factoid answers from given context, only use your own knowledge when answer cannot be found in the context.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] «ozone depletion, gradual thinning of Earth’s ozone layer in the upper atmosphere caused by the release of chemical compounds containing gaseous chlorine or bromine from industry and other human activities. The thinning is most pronounced in the polar regions, especially over Antarctica.»
[2] «Ozone depletion is a major environmental problem because it increases the amount of ultraviolet (UV) radiation that reaches Earth’s surface, which increases the rate of skin cancer, eye cataracts, and genetic and immune system damage. The Montreal Protocol, ratified in 1987, was the first of several comprehensive international agreeme

Note: if you decide to use `BootstrapFewShot` on this, be sure not to use the metric we defined above, which requires that the passage embeds the correct answer as a substring. Now that we are summarizing, this is unlikely to hold, even if the answers are good ones.

## Question 4: Your original system [3 points]

This question asks you to design your own few-shot OpenQA system. All of the code above can be used and modified for this, and the requirement is just that you try something new that goes beyond what we've done so far. 

Terms for the bake-off:

* You can make free use of SQuAD and other publicly available data.

* The LM must be an autoregressive language model. No trained QA components can be used. This includes general purpose LMs that have been fine-tuned for QA. (We have obviously waded into some vague territory here. The spirit of this is to make use of frozen, general-purpose models. We welcome questions about exactly how this is defined, since it could be instructive to explore this.)

Here are some ideas for the original system:

* We have relied almost entirely on `dspy.Predict`. Drop-in replacements include `dspy.ChainOfThought` and `dspy.ReAct`.

* We have used only one retriever. DSPy supports other retrieval mechanisms, including retrieval using [You.com](https://you.com/).

* DSPy includes additional optimizers. Two that are worth trying are `SignatureOptimizer` for automatic prompt exploration and `BootstrapFewShotWithRandomSearch`, which combines `LabeledFewShot` and `BootstrapFewShot`,

* Our one-step summarization procedure from Question 3 doesn't change the query to the retriever. We might want it to change as we gather evidence. This is a common design principle for multi-hop OpenQA systems.

__Original system instructions__:

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# START COMMENT: Enter your system description in this cell.

"""
Textual description of your system:

The idea is to combine the information retrieval of ColBERT and You.com get a larger number of candidates (20 results from each in this case), 
and use a cross encoder reranker to pick 10 most relevant passages from the 40, and then pass to the QA module.
The QA module also has some minor improvement, I updated the prompt a little bit and also using the ChainOfThought module so that the LLM 
will reasoning before answering.
Overall after some experiment, I found the ColBERT mostly return more relevant passage than the you.com, including results from 
you.com's search API to cover some knowledge gap for ColBERT server.
"""

from dspy.retrieve.you_rm import YouRM
from sentence_transformers import CrossEncoder
import os

class EnhancedQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers from given context, only use your own knowledge when answer cannot be found in the context."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

    
class EnhancedRAG(dspy.Module):
    def __init__(self, num_passages=10, colbert_cnt=20, you_cnt=20):
        super().__init__()
        self.num_passages = num_passages
        self.colbert_cnt = colbert_cnt
        self.you_cnt = you_cnt
        self.colbert_retrieve = dspy.ColBERTv2(url="http://10.0.0.38:8889//api/search")
        self.you_retrieve = YouRM(endpoint="search", safesearch="strict")
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        self.generate_answer = dspy.ChainOfThought(EnhancedQASignature)

    def forward(self, question):
        colbert_results = [r['long_text'] for r in self.colbert_retrieve(question, k=self.colbert_cnt)]
        you_results = [r['long_text'] for r in self.you_retrieve(question, k=self.you_cnt)]
        reranked_context = [r['text'] for r in self.reranker.rank(question, colbert_results + you_results, return_documents=True)][:self.num_passages]
        prediction = self.generate_answer(context=reranked_context, question=question)
        return dspy.Prediction(context=reranked_context, answer=prediction.answer)

enhanced_rag = EnhancedRAG()
# STOP COMMENT: Please do not remove this comment.

In [227]:
enhanced_rag = EnhancedRAG()

In [230]:
en_rag('what are the colors for iPhone 16 Pro Max?')

Prediction(
    context=['The iPhone 16 in White, a new color. Apple/Business Insider ... The iPhone 16 Pro and iPhone 16 Pro Max are available in four colors: Desert Titanium, Natural Titanium, Black Titanium, and White Titanium. Their titanium frames and rear glass panels have a frosted matte texture.', "The iPhone 16 also comes with a new color-infused backglass that has allowed Apple to get some gorgeous, saturated, and rich colors like Ultramarine, Teal, and Pink. On the other hand, the iPhone 16 Pro and Pro Max continue Apple's tradition of a more conservative color approach for the pro-oriented people.", 'Black color is almost identical on the iPhone 16 and iPhone 15 | Image Credit - Apple ... The iPhone 16 Pro and Pro Max maintain a professional and sleek aesthetic, with subtle and refined color options. The Pro models this year continue to follow the Titanium-inspired theme, but this time, they feature a grade 5 titanium.', 'For the Black Titanium version, the camera island an

In [232]:
en_rag("what's the return policy for iPhone 16 Pro?")

Prediction(
    context=["Refund on Apple iPhone 15 and iPhone 14: What Apple's 'price protection' policy says and the conditions that apply ... Apple has unveiled its new iPhone 16 series featuring four models: iPhone 16, iPhone 16 Plus, iPhone 16 Pro, and iPhone 16 Pro Max. Pre-orders commence on September 13 with sales starting on September 20.", 'Returning after 14 days Can I return my sealed iPhone 12 Pro Max after 14 days 2177 2 ... According to Apples Returns and Refunds policy below the iPhone will likely have been required to be returned within 14 calendar days.', 'I purchased the iphone 16 plus a few days back, is it possible to go to the apple store and switch/upgrade to the pro max? ... Click here to read about Apple Store Returns & Refunds--> Returns & Refunds - Shopping Help - Apple - For the USA "You have 14 calendar days to return an item from the date you received it."', "Some people are returning because they think there are flaws in the 16 Pro though. So far there's 

## Question 5: Bakeoff entry [1 point]

For the bake-off, you simply need to be able to run your system on the file 

```data/openqa/cs224u-openqa-test-unlabeled.txt```

The following code should download it for you if necessary:

In [150]:
import wget

if not os.path.exists(os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")):
    os.makedirs(os.path.join('data', 'openqa'), exist_ok=True)
    wget.download('https://web.stanford.edu/class/cs224u/data/cs224u-openqa-test-unlabeled.txt', out='data/openqa/')

100% [..............................................................................] 16822 / 16822

If the above fails, you can just download https://web.stanford.edu/class/cs224u/data/cs224u-openqa-test-unlabeled.txt and place it in `data/openqa`.

This file contains only questions. The starter code below will help you structure this. It writes a file "cs224u-openqa-bakeoff-entry.json" to the current directory. That file should be uploaded as-is. Please do not change its name.

In [228]:
import json
import tqdm

def create_bakeoff_submission(model):
    """"
    The argument `model` is a `dspy.Module`. The return value of its
    `forward` method must have an `answer` attribute.
    """

    filename = os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")

    # This should become a mapping from questions (str) to response
    # dicts from your system.
    gens = {}

    with open(filename) as f:
        questions = f.read().splitlines()

    # Here we loop over the questions, run the system `model`, and
    # store its `answer` value as the prediction:
    for question in tqdm.tqdm(questions):
        gens[question] = model(question=question).answer

    # Quick tests we advise you to run:
    # 1. Make sure `gens` is a dict with the questions as the keys:
    assert all(question in gens for q in questions)
    # 2. Make sure the values are str:
    assert all(isinstance(d, str) for d in gens.values())

    # And finally the output file:
    with open("cs224u-openqa-bakeoff-entry.json", "wt") as f:
        json.dump(gens, f, indent=4)

Here's what it looks like to evaluate our first program, `basic_qa_model`, on the bakeoff data:

In [229]:
create_bakeoff_submission(enhanced_rag)

100%|█████████████████████████████████████████████████████████████| 400/400 [11:06<00:00,  1.67s/it]
